## Let's build and evaluate agents using Smolagents

If you haven't installed `smolagents` yet, you can do so by running the following command:


In [7]:
!pip install smolagents -U
!pip install duckduckgo-search


In [8]:
from huggingface_hub import notebook_login

notebook_login()

## Finding books for implementing AI-ML in the workplace, using `smolagents`

For the model, we'll rely on `InferenceClientModel`, which provides access to Hugging Face's [Inference API](https://huggingface.co/docs/api-inference/index). The default model is `"Qwen/Qwen2.5-Coder-32B-Instruct"`, which is performant and available for fast inference, but you can select any compatible model from the Hub.

Running an agent is quite straightforward:


In [10]:

from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=InferenceClientModel(provider="together"))

agent.run("Search for the best book recommendations for implementing AI in the workplace.")

ImportError: You must install package `ddgs` to run this tool: for instance run `pip install ddgs`.

Let's revisit the previous example, but this time we'll use a `ToolCallingAgent` to highlight the difference. We'll build an agent that can search the web using DuckDuckGo, just like in our Code Agent example. The only difference is the agent type - the framework handles everything else:


In [6]:
!pip install duckduckgo-search

from smolagents import ToolCallingAgent, DuckDuckGoSearchTool, InferenceClientModel

agent = ToolCallingAgent(tools=[DuckDuckGoSearchTool()], model=InferenceClientModel(provider = "together"))

agent.run("Search for the best book recommendations for implementing AI in the workplace.")

ImportError: You must install package `ddgs` to run this tool: for instance run `pip install ddgs`.

When you examine the agent's trace, instead of seeing `Executing parsed code:`, you'll see something like:

```text
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': "best music recommendations for a party at Wayne's         │
│ mansion"}                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
```

The agent generates a structured tool call that the system processes to produce the output, rather than directly executing code like a `CodeAgent`.

Now that we understand both agent types, we can choose the right one for our needs. Let's continue exploring `smolagents` to make Alfred's party a success! 🎉


### Inspecting Our Agent with OpenTelemetry and Langfuse 📡

Agents, by nature, are unpredictable and difficult to inspect.

Once again, `smolagents` comes to the rescue! It embraces the [OpenTelemetry](https://opentelemetry.io/) standard for instrumenting agent runs, allowing seamless inspection and logging. With the help of [Langfuse](https://langfuse.com/) and the `SmolagentsInstrumentor`, we can easily track and analyze the agent’s behavior.

Setting it up is straightforward!

First, we need to install the necessary dependencies:


In [3]:
!pip install smolagents[telemetry] opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.4/247.4 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.3 MB/s eta 0:00:00


Next, Alfred has already created an account on Langfuse and has his API keys ready. If you haven’t done so yet, you can sign up for Langfuse Cloud [here](https://cloud.langfuse.com/) or explore [alternatives](https://huggingface.co/docs/smolagents/tutorials/inspect_runs).

Once you have your API keys, they need to be properly configured as follows:


In [ ]:
import base64
import os

os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_SECRET_KEY"] = ""

LANGFUSE_PUBLIC_KEY = os.environ.get("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.environ.get("LANGFUSE_SECRET_KEY")

if not LANGFUSE_PUBLIC_KEY or not LANGFUSE_SECRET_KEY:
    raise ValueError("Langfuse public or secret keys are missing!")

LANGFUSE_AUTH = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

#os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

print("Langfuse environment variables set successfully!")



Langfuse environment variables set successfully!


Finally, Alfred is ready to initialize the `SmolagentsInstrumentor` and start tracking his agent's performance.


In [5]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

Alfred is now connected 🔌! The runs from `smolagents` are being logged in Langfuse, giving him full visibility into the agent's behavior. With this setup, he's ready to revisit previous runs and refine his Party Preparator Agent even further.


In [13]:
from smolagents import CodeAgent, InferenceClientModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=InferenceClientModel(provider = "together"))

agent.run("Search for the best book recommendations for implementing AI in the workplace.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best book recommendations for implementing AI in the workplace.                                  │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  book_recommendations = web_search(query="best books for implementing AI in the workplace")                       
  print(book_recommendations)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[The 10 Best Books on AI & HR That Every Leader Must 
Read](https://friendlychro.com/2025/07/22/best-books-ai-hr-leaders/)
Jul 22, 2025 · These ten groundbreaking books go beyond buzzwords to deliver deep insights, ethical frameworks, and
visionary strategies for leveraging AI in talent management, workforce planning, and leadership.

[9 standout books on AI and tech - McKinsey & 
Company](https://www.mckinsey.com/featured-insights/themes/9-standout-books-on-ai-and-tech)
Jul 16, 2025 · As gen AI accelerates this shift, it’s never been more important to understand the broader landscape
of technology. To help you stay informed, explore nine must-read books recommended by leaders in technology, 
publishing, the public sector, and more.

[10 Best AI Books Every Business Leader Should Read - 
LinkedIn](https://www.linkedin.com/pulse/10-best-ai-books-every-business-leader-should-read-sarah-socha-oufde)
Jun 10, 2025 · To help you stay ahead I have compiled a list of 10 essential AI books that offer practical 
insights, strategic frameworks, and visionary thinking for business leaders.

[The Best Books on Artificial Intelligence (AI) to Read in 2025](https://www.qualtrics.com/blog/books-on-ai/)
Dec 4, 2024 · Hunting for the best books on artificial intelligence? Look no further; our curated list of 
must-reads covers essentials for everyone – from beginners to seasoned AI pros. We’ve compiled books spanning 10 
topics exploring AI 's history, ethical considerations, future, and more.

[10 Must-Read Books on AI Recommended by Thought Leaders Images Great Books on AI for Business Leaders - CEOWORLD 
magazine The 11 Best AI Books in 2025 [Updated Guide] | Beginner to 
Pro](https://medium.com/@rpatech.ai/10-must-read-books-on-ai-recommended-by-thought-leaders-b3eaecb2bf43)
Jan 21, 2025 · Here are some books on AI that are recommended by technology leaders like Lila Ibrahim, COO of 
Google DeepMind, Amy Webb, founder and CEO of Future Today Institute, and others. View all Apr 13, 2025 · These 
five books all look at the best ways to harness AI and data for businesses and beyond, and provide business leaders
with a vision for navigating the present as well as the future. AI Books - Here we have listed a few basic and 
advanced Artificial Intelligence books , which will help you find your way around AI .

[Great Books on AI for Business Leaders - CEOWORLD 
magazine](https://ceoworld.biz/2025/04/13/great-books-on-ai-for-business-leaders/)
Apr 13, 2025 · These five books all look at the best ways to harness AI and data for businesses and beyond, and 
provide business leaders with a vision for navigating the present as well as the future.

[The 11 Best AI Books in 2025 [Updated Guide] | Beginner to 
Pro](https://hackr.io/blog/artificial-intelligence-books)
AI Books - Here we have listed a few basic and advanced Artificial Intelligence books , which will help you find 
your way around AI .

[6 Books on AI in the 
Workplace](https://www.publishersweekly.com/pw/by-topic/new-titles/adult-announcements/article/95171-six-books-on-a
i-in-the-workplace.html)
Implementing AI practices in a workplace needs to be more than good for the bottom line: it should also be done 
carefully and thoughtfully, says ...

[[PDF] Handbook Of Artificial Intelligence At Work - 
eBooks](https://ardhindie.com/pdf/handbook-of-artificial-intelligence-at-work)
Written by prominent thought leaders in the global fintech space, The AI Book aggregates diverse expertise into a 
single, informative volume and ...

[[PDF] The Ai Powered Workplace - eBooks Review](https://ardhindie.com/pdf/the-ai-powered-workplace)
Download The Ai Powered Workplace PDF/ePub or read online books in Mobi eBooks. ... The Ai Powered Workplace 
written by Ronald Ashri and has been ...

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(refined_book_titles)                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: ['The Algorithm: How AI Decides Who Gets Hired, Monitored, and Fired', 'The Ai Powered Workplace', 
'Handbook Of Artificial Intelligence At Work', 'Working with AI']

[Step 7: Duration 1.25 seconds| Input tokens: 28,208 | Output tokens: 855]

['The Algorithm: How AI Decides Who Gets Hired, Monitored, and Fired',
 'The Ai Powered Workplace',
 'Handbook Of Artificial Intelligence At Work',
 'Working with AI']

Alfred can now access this logs [here](https://cloud.langfuse.com/project/cm7bq0abj025rad078ak3luwi/traces/995fc019255528e4f48cf6770b0ce27b?timestamp=2025-02-19T10%3A28%3A36.929Z) to review and analyze them.

Meanwhile, the [suggested playlist](https://open.spotify.com/playlist/0gZMMHjuxMrrybQ7wTMTpw) sets the perfect vibe for the party preparations. Cool, right? 🎶


In [ ]:
agent = ToolCallingAgent(tools=[DuckDuckGoSearchTool()], model=InferenceClientModel(provider = "together"))

agent.run("Search for the best book recommendations for implementing AI in the workplace.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best book recommendations for implementing AI in the workplace.                                  │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': 'best book recommendations for implementing AI in the      │
│ workplace'}                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: ## Search Results

|6 Books on AI in the Workplace - Publishers 
Weekly](https://www.publishersweekly.com/pw/by-topic/new-titles/adult-announcements/article/95171-six-books-on-ai-i
n-the-workplace.html)
Implementing AI practices in a workplace needs to be more than good for the bottom line: it should also be done 
carefully and thoughtfully, says Gambelin, who founded the AI Ethics advisory firm ...

|10 Must-Read Books on AI Recommended by Thought 
Leaders](https://medium.com/@rpatech.ai/10-must-read-books-on-ai-recommended-by-thought-leaders-b3eaecb2bf43)
Here are some books on AI that are recommended by technology leaders like Lila Ibrahim, COO of Google DeepMind, Amy
Webb, founder and CEO of Future Today Institute, and others.

|What to read next: books on AI - McKinsey & 
Company](https://www.mckinsey.com/featured-insights/themes/what-to-read-next-books-on-ai)
Here are six books recommended by leaders, including McKinsey's Yuval Atsmon and Lareina Yee. For more suggestions 
from 50-plus global executives, editors in chief, and McKinsey leaders, check out " What to read next: McKinsey's 
2024 annual book recommendations." AI Needs You: How We Can Change AI's Future and Save Our Own by Verity ...

|The Best Books on Artificial Intelligence (AI) to Read in 2025](https://www.qualtrics.com/blog/books-on-ai/)
Discover the top 10 best books on artificial intelligence that provide valuable insights and knowledge. Click 
though to genuinely gain a detailed understanding on AI.

|26 Best Books on AI in 2025 (65+ Reviewed)](https://usefulai.com/books)
Discover the top 26 AI books for 2025, from technological advancements to ethical dilemmas. Over 65 books reviewed 
to guide you through the AI revolution.

|AI and the Future of Work: 7 Must-Read Books - Analytics 
Insight](https://www.analyticsinsight.net/ampstories/artificial-intelligence/ai-and-the-future-of-work-7-must-read-
books)
Discover 7 essential books on AI's impact on work, offering insights and strategies to thrive in the evolving job 
landscape.

|The best books on AI and the future of work - Shepherd](https://shepherd.com/best-books/ai-and-the-future-of-work)
The first comprehensive book on new tech and its impacts, following big steps made in AI progress in the early 
2010s. The authors bring home the point that we are undergoing a watershed moment as tools no longer substitute 
merely for physical labor encroach on mental tasks - hence the book's title. After centuries of fleeing blue-collar
jobs to take refuge in cerebral work, we are being ...

|22 Best Books on AI and Machine Learning in 2025 (Top Rated)](https://aimojo.io/books-ai-and-machine-learning/)
Elevate your understanding of AI with our selection of the 22 best books on AI and Machine Learning for 2025. 
Discover groundbreaking concepts and innovative techniques.

|4 Great Books on How to Implement AI in Business, Automation, and the 
...](https://globisinsights.com/future-of-work/books-on-how-to-implement-ai-in-business/)
GLOBIS University faculty discuss their favorite books on how to implement AI in business, automation, and the 
future of work.

|12 Best Books on Artificial Intelligence (AI) You Can't Miss](https://maven.com/articles/best-books-on-ai)
And if you already have a basic understanding of AI, work in artificial intelligence, or are in a line of work that
may be affected by it, there's even more reason to stay on top of the field. Here's a list of AI book 
recommendations suitable for beginners, intermediates, and experts.

[Step 1: Duration 1.66 seconds| Input tokens: 1,188 | Output tokens: 31]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: Top 5 Book Recommendations for Implementing AI in the Workplace:   │
│                                                                                                                 │
│ 1. **'AI Needs You: How We Can Change AI's Future and Save Our Own' by Verity Harding**                         │
│ 2. **'AI and the Future of Work: 7 Must-Read Books' by Analytics Insight**                                      │
│ 3. **'The Best Books on Artificial Intelligence (AI) to Read in 2025' by Qualtrics**                            │
│ 4. **'The Best Books on AI and the Future of Work' by Shepherd**                                                │
│ 5. **'22 Best Books on AI and Machine Learning in 2025' by AI Mojo**                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Top 5 Book Recommendations for Implementing AI in the Workplace:

1. **'AI Needs You: How We Can Change AI's Future and Save Our Own' by Verity Harding**
2. **'AI and the Future of Work: 7 Must-Read Books' by Analytics Insight**
3. **'The Best Books on Artificial Intelligence (AI) to Read in 2025' by Qualtrics**
4. **'The Best Books on AI and the Future of Work' by Shepherd**
5. **'22 Best Books on AI and Machine Learning in 2025' by AI Mojo**

[Step 2: Duration 6.14 seconds| Input tokens: 3,311 | Output tokens: 539]

"Top 5 Book Recommendations for Implementing AI in the Workplace:\n\n1. **'AI Needs You: How We Can Change AI's Future and Save Our Own' by Verity Harding**\n2. **'AI and the Future of Work: 7 Must-Read Books' by Analytics Insight**\n3. **'The Best Books on Artificial Intelligence (AI) to Read in 2025' by Qualtrics**\n4. **'The Best Books on AI and the Future of Work' by Shepherd**\n5. **'22 Best Books on AI and Machine Learning in 2025' by AI Mojo**"